In [1]:
pip install tqdm notebook==7.1.2 groq elasticsearch pandas scikit-learn ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tokenizers

In [3]:
!pip install requests

In [4]:
from transformers import AutoTokenizer

In [5]:
import json

In [6]:
import os

os.environ["GROQ_API_KEY"] = "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"

from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from elasticsearch import Elasticsearch

In [10]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
es_client.info()

ObjectApiResponse({'name': 'e340dc286eea', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2MwdrOn4Tuukehbh95NdJQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name,body=index_settings)

In [13]:
from tqdm.auto import tqdm

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [62]:
query = 'How do I execute a command in a running docker container?'

In [63]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    top_score = None

    for i, hit in enumerate(response['hits']['hits']):
        result_docs.append(hit['_source'])
        if i == 0:  # Capture the score of the top-ranking result
            top_score = hit['_score']

    return result_docs, top_score

In [64]:
documents, top_score = elastic_search(query)

In [65]:
print(f"The score for the top ranking result is: {top_score}")

The score for the top ranking result is: 84.17781


In [71]:
if len(documents) >= 3:
    third_question = documents[2]['question']
    print(f"The 3rd question returned by the search engine is: {third_question}")
else:
    print("Less than 3 results found.")

The 3rd question returned by the search engine is: How do I copy files from my local machine to docker container?


In [66]:
context_template = """
Q: {question}
A: {text}
""".strip()
    
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
    
QUESTION: {question}
    
CONTEXT:
{context}
""".strip()

In [67]:
def build_prompt(query, search_results):
    context_entries = []
    for result in search_results:
        context_entry = context_template.format(question=result['question'], text=result['text'])
        context_entries.append(context_entry)
    
    context = "\n\n".join(context_entries)
    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [68]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
    )
    return response.choices[0].message.content

In [69]:
def rag(query):
    search_results = documents
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [70]:
rag(query)

'To execute a command in a running Docker container, you can use the following command:\n\ndocker exec -it <container-id> bash\n\nReplace <container-id> with the ID of the running container that you find using the command:\n\ndocker ps'

In [76]:
# Calculate the length of the resulting prompt
prompt_length = len(prompt)
print("Length of the resulting prompt:", prompt_length)

Length of the resulting prompt: 1407


In [73]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [ ]:
prompt = build_prompt(query, documents)

# Calculate the number of tokens
tokens = tokenizer.encode(prompt)
prompt_length = len(tokens)